<a href="https://colab.research.google.com/github/Emmanuelprime/Emmanuelprime/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re
import json
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem.porter import PorterStemmer
np.random.seed(42)


Declaring variable


In [2]:
vocabulary_size = 10000
feature_dimension = 40
sentence_length = 50
sentences =[]
labels =[]


Downloading the dataset and reading the dataset

In [3]:
with open('sca.json') as f:
  sentences_from_data = json.load(f)
  for items in sentences_from_data:
    sentences.append(items['headline'])
    labels.append(items['is_sarcastic'])


Preprocessing of the text


In [4]:
from nltk.lm.preprocessing import pad_sequence
ps = PorterStemmer()
final_result = []
for i in range(0,len(sentences)):
  review = re.sub('[^a-zA-Z]',' ',sentences[i])
  review = review.lower()
  #review = review.split()
  #review = [ps.stem(word)for word in review if not word in stopwords.words('english')]
  #review = ' '.join(review)
  final_result.append(review)

one_hot_rep = [one_hot(words,vocabulary_size)for words in final_result]
sequenced_data = pad_sequences(one_hot_rep,padding ='post',maxlen = sentence_length)

Creating train and test data

In [5]:
from sklearn.model_selection import train_test_split
X = np.array(sequenced_data)
Y = np.array(labels)

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

Building the model

In [6]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_size,feature_dimension,input_length=sentence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100)),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1,activation ='sigmoid')
])

model2 =tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_size,feature_dimension,input_length=sentence_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64,activation = 'relu'),
    tf.keras.layers.Dense(1, activation='relu')
])

compiling the model

In [7]:
model1.compile(optimizer='adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
model2.compile(optimizer = 'adam',loss ='binary_crossentropy',metrics =['accuracy'])

fit the model

In [8]:
hist1 = model1.fit(x_train,y_train,epochs =10,validation_data=(x_test,y_test),verbose=0)
hist2 = model2.fit(x_train,y_train,epochs = 10,validation_data=(x_test,y_test),verbose = 0)




In [14]:
df_hist1 = pd.DataFrame(hist1.history)
df_hist2 = pd.DataFrame(hist2.history)
epochs =10

print(df_hist1)
print(df_hist2)

       loss  accuracy  val_loss  val_accuracy
0  0.419652  0.795058  0.369967      0.834893
1  0.253230  0.897693  0.382698      0.838637
2  0.166522  0.939720  0.477796      0.829277
3  0.113294  0.960547  0.550705      0.824598
4  0.071028  0.977021  0.639090      0.813179
5  0.047280  0.984462  0.776581      0.812617
6  0.028856  0.990031  0.922197      0.812430
7  0.021730  0.992418  1.057928      0.809247
8  0.013583  0.995320  1.317217      0.806252
9  0.010577  0.996537  1.385911      0.807001
       loss  accuracy  val_loss  val_accuracy
0  0.605434  0.726073  0.469468      0.827031
1  0.416265  0.853746  0.481892      0.839199
2  0.378497  0.875649  0.494404      0.832273
3  0.645588  0.861188  0.576851      0.819543
4  0.320354  0.892685  0.589275      0.827031
5  0.284323  0.908972  0.834987      0.824410
6  0.376858  0.891094  0.874399      0.823662
7  1.030740  0.876773  6.465372      0.566080
8  4.470323  0.643469  2.594527      0.716773
9  0.855093  0.823326  0.860989   